In [ ]:
# !pip install torch torchvision torchaudio
# !pip install torch-geometric
!pip install torch torchvision torchaudio torch-geometric scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.0 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd

In [ ]:
!pip install gdown
!gdown --no-check-certificate --folder https://drive.google.com/drive/folders/17g55PHmMWFo6aBNhmzjOxVDfwtSDFUl3?usp=drive_link
# !gdown --no-check-certificate --folder https://drive.google.com/drive/folders/1Uf9L2NuLPFcnr0ydykXHnBQKE5ofKCBU?usp=drive_link

Retrieving folder contents
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'drive.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
Processing file 1J2eOhACVhee6fnrO0wA5ct255sVR5EQS BCNET_regular.csv
Processing file 1d4e4g7PNVkxO-H4ZknemlWZNnWOpJkLV Code_Red_I.csv
Processing file 1IleVfZkR-EQ0X6-TmM1UXZhDphFn-auG Moscow_blackout.csv
Processing file 1YFeW7-KnN474mkULMOAs3h3cxFS7tdb4 Nimda.csv
Processing file 1_PlE4ABTghvpYQ7M1MfT8w5xQmXopQok RIPE_regular.csv
Processing file 1WjlwJW4dHABkkx0ANAfAQBLIo9OZf5C1 Slammer.csv
Processing file 1RKRYfuaEmRCsgUaHIZICxL9FngrMYP7u WannaCrypt.csv
Retrieving folder contents completed
Building directory structure
Building directory structure completed
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unv

In [ ]:
data = pd.read_csv("data_full.csv")

In [ ]:
data.head()

,start_hour,start_minute,start_second,asn,num_announcements,num_withdrawn_routes,withdrawn_routes_length,num_withdrawn_nlri_prefixes,avg_as_path_length,attack_label
0,3,0,0,20912,152,107,228,14,2,1
1,0,12,33,6447,195,4,76,6,2,0
2,8,45,0,6447,59,74,78,26,4,1
3,8,15,0,6447,175,76,262,32,3,1
4,19,0,0,20912,195,93,213,10,5,1


### MvTS

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np

# Load and preprocess data
data_path = 'data_full.csv'
data = pd.read_csv(data_path)

# Extract features and labels
asn_nodes = [6447, 2914, 20912]
features = []
labels = []

for asn in asn_nodes:
    asn_data = data[data['asn'] == asn]
    node_features = asn_data.drop(columns=['attack_label', 'asn']).values
    node_labels = asn_data['attack_label'].iloc[0]  # Use the first label for the entire node

    features.append(torch.tensor(node_features, dtype=torch.float))
    labels.append(torch.tensor(node_labels, dtype=torch.long))  # Single label per node

# Ensure equal size for all feature tensors by padding
max_len = max(f.size(0) for f in features)
padded_features = []
for node_features in features:
    padded = torch.nn.functional.pad(
        node_features,
        (0, 0, 0, max_len - node_features.size(0)),  # Pad time dimension
        value=0  # Padding value
    )
    padded_features.append(padded)

# Stack tensors for input
features = torch.stack(padded_features, dim=0)  # Shape: [num_nodes, max_len, num_features]
labels = torch.tensor(labels, dtype=torch.long)  # Shape: [num_nodes]

# Define Transformer model
class TimeSeriesTransformer(nn.Module):
    def __init__(self, input_dim, num_heads, num_layers, hidden_dim, output_dim):
        super(TimeSeriesTransformer, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim)
        self.transformer = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        # x shape: [num_nodes, max_len, num_features]
        x = self.transformer(x.permute(1, 0, 2))  # Transpose to [max_len, num_nodes, num_features]
        x = x.mean(dim=0)  # Global average pooling over time dimension
        x = self.fc(x)  # Final classification layer
        return x

# Initialize model
input_dim = features.size(-1)
num_heads = 4
num_layers = 2
hidden_dim = 128
output_dim = 2
model = TimeSeriesTransformer(input_dim, num_heads, num_layers, hidden_dim, output_dim)

# Define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Train the model
num_epochs = 50
for epoch in range(num_epochs):
    model.train()

    # Forward pass
    outputs = model(features)
    loss = criterion(outputs, labels)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}")

# Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(features)
    predictions = torch.argmax(outputs, dim=1).cpu().numpy()
    true_labels = labels.cpu().numpy()

    # Print classification report
    print("\nClassification Report:")
    print(classification_report(true_labels, predictions))

# Save the trained model
torch.save(model.state_dict(), 'time_series_transformer.pth')

print("Model training and evaluation complete.")


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1/50, Loss: 0.7416
Epoch 2/50, Loss: 0.6339
Epoch 3/50, Loss: 0.5863
Epoch 4/50, Loss: 0.5596
Epoch 5/50, Loss: 0.5611
Epoch 6/50, Loss: 0.5739
Epoch 7/50, Loss: 0.5837
Epoch 8/50, Loss: 0.5879
Epoch 9/50, Loss: 0.5712
Epoch 10/50, Loss: 0.5508
Epoch 11/50, Loss: 0.5548
Epoch 12/50, Loss: 0.5273
Epoch 13/50, Loss: 0.5200
Epoch 14/50, Loss: 0.5062
Epoch 15/50, Loss: 0.4979
Epoch 16/50, Loss: 0.4970
Epoch 17/50, Loss: 0.4811
Epoch 18/50, Loss: 0.4907
Epoch 19/50, Loss: 0.4710
Epoch 20/50, Loss: 0.4600
Epoch 21/50, Loss: 0.4638
Epoch 22/50, Loss: 0.4336
Epoch 23/50, Loss: 0.4304
Epoch 24/50, Loss: 0.4288
Epoch 25/50, Loss: 0.3993
Epoch 26/50, Loss: 0.4027
Epoch 27/50, Loss: 0.3983
Epoch 28/50, Loss: 0.3859
Epoch 29/50, Loss: 0.3709
Epoch 30/50, Loss: 0.3514
Epoch 31/50, Loss: 0.3474
Epoch 32/50, Loss: 0.3082
Epoch 33/50, Loss: 0.2892
Epoch 34/50, Loss: 0.2823
Epoch 35/50, Loss: 0.2582
Epoch 36/50, Loss: 0.2350
Epoch 37/50, Loss: 0.2123
Epoch 38/50, Loss: 0.2082
Epoch 39/50, Loss: 0.